In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
import os
from datetime import datetime
from functools import reduce
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error as mae, mean_squared_error
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, TimeDistributed, Conv1D, MaxPooling1D, Flatten, Bidirectional, Input, Flatten, Activation, Reshape, RepeatVector, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

In [ ]:
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

2022-04-30 05:03:31.555579: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-30 05:03:31.618903: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-30 05:03:31.707073: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-30 05:03:31.708281: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0

Num GPUs Available:  1


2022-04-30 05:03:33.667085: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-30 05:03:33.668682: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-30 05:03:33.670076: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-30 05:03:33.671955: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15403 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
2022-04-30 05:03:33.678253: I tensorflow/stream_executor/cuda/cuda_gpu_executor.

In [ ]:
train_df = pd.read_csv('../input/tabular-playground-series-mar-2022/train.csv', parse_dates=["time"])
test_df = pd.read_csv('../input/tabular-playground-series-mar-2022/test.csv', parse_dates=["time"])
train_df.dropna(inplace = True)
test_df.dropna(inplace = True)

In [ ]:
train_df.info()

In [ ]:
train_df

,row_id,time,x,y,direction,congestion
0,0,1991-04-01 00:00:00,0,0,EB,70
1,1,1991-04-01 00:00:00,0,0,NB,49
2,2,1991-04-01 00:00:00,0,0,SB,24
3,3,1991-04-01 00:00:00,0,1,EB,18
4,4,1991-04-01 00:00:00,0,1,NB,60
...,...,...,...,...,...,...
848830,848830,1991-09-30 11:40:00,2,3,NB,54
848831,848831,1991-09-30 11:40:00,2,3,NE,28
848832,848832,1991-09-30 11:40:00,2,3,SB,68
848833,848833,1991-09-30 11:40:00,2,3,SW,17


In [ ]:
train_pv = train_df.pivot(index=['time'], columns=['x', 'y', 'direction'], values= 'congestion')
train_pv

x                     0                                      ...   2          \
y                     0           1               2          ...   2           
direction            EB  NB  SB  EB  NB  SB  WB  EB  NB  SB  ...  SB  SE  SW   
time                                                         ...               
1991-04-01 00:00:00  70  49  24  18  60  58  26  31  49  46  ...  36  51  47   
1991-04-01 00:20:00  70  49  24  26  64  55  57  41  40  46  ...  10  51  50   
1991-04-01 00:40:00  70  49  24  31  51  57  91  42  41  35  ...  32  51  49   
1991-04-01 01:00:00  70  49  24  36  51  46  49  39  44  58  ...  60  51  49   
1991-04-01 01:20:00  70  49  24  29  53  83  62  36  46  46  ...  43  51  49   
...                  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ...  ..  ..  ..   
1991-09-30 10:20:00  43  29  60  23  63  61  42  47  51  56  ...  66  20  50   
1991-09-30 10:40:00  47  28  35  24  62  63  32  44  57  65  ...  60  21  49   
1991-09-30 11:00:00  40  31  53  22  65  58  38  44  60  62  ...  58  27  53   
1991-09-30 11:20:00  52  36  59  35  67  54  26  44  59  57  ...  52  20  53   
1991-09-30 11:40:00  70  40  51  24  65  52  42  43  62  77  ...  64  40  51   

x                                                
y                         3                      
direction            WB  EB  NB  NE  SB  SW  WB  
time                                             
1991-04-01 00:00:00  51  39  64  30  70  29  26  
1991-04-01 00:20:00  58  33  67  30  80  29  57  
1991-04-01 00:40:00  63  33  64  30  79  29  48  
1991-04-01 01:00:00  52  33  33  30  77  29  44  
1991-04-01 01:20:00  50  33  50  30  79  29  44  
...                  ..  ..  ..  ..  ..  ..  ..  
1991-09-30 10:20:00  70  90  53  29  68  22  49  
1991-09-30 10:40:00  68  45  52  25  70  20  47  
1991-09-30 11:00:00  68  44  58  29  69  19  37  
1991-09-30 11:20:00  67  57  59  22  69  22  53  
1991-09-30 11:40:00  71  48  54  28  68  17  24  

[13059 rows x 65 columns]

In [ ]:
full_data = pd.DataFrame(pd.date_range('1991-04-01 00:00:00', '1991-09-30 11:40:00', freq='20Min'), columns=['time'])
full_data.index = full_data['time']
full_data.columns = pd.MultiIndex.from_product([full_data.columns, [''],['']])

In [ ]:
full_train_pv = pd.merge(train_pv,full_data,how='outer', left_index=True, right_index=True)
full_train_pv.drop('time',axis=1,inplace=True)

Imputing Null values

In [ ]:
from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
imputed = imp_mean.fit_transform(full_train_pv)
imputed_train = pd.DataFrame(imputed,columns=full_train_pv.columns,index=full_train_pv.index).astype(int)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,


In [ ]:
imputed_train

x                     0                                      ...   2          \
y                     0           1               2          ...   2           
direction            EB  NB  SB  EB  NB  SB  WB  EB  NB  SB  ...  SB  SE  SW   
time                                                         ...               
1991-04-01 00:00:00  70  49  24  18  60  58  26  31  49  46  ...  36  51  47   
1991-04-01 00:20:00  70  49  24  26  64  55  57  41  40  46  ...  10  51  50   
1991-04-01 00:40:00  70  49  24  31  51  57  91  42  41  35  ...  32  51  49   
1991-04-01 01:00:00  70  49  24  36  51  46  49  39  44  58  ...  60  51  49   
1991-04-01 01:20:00  70  49  24  29  53  83  62  36  46  46  ...  43  51  49   
...                  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ...  ..  ..  ..   
1991-09-30 10:20:00  43  29  60  23  63  61  42  47  51  56  ...  66  20  50   
1991-09-30 10:40:00  47  28  35  24  62  63  32  44  57  65  ...  60  21  49   
1991-09-30 11:00:00  40  31  53  22  65  58  38  44  60  62  ...  58  27  53   
1991-09-30 11:20:00  52  36  59  35  67  54  26  44  59  57  ...  52  20  53   
1991-09-30 11:40:00  70  40  51  24  65  52  42  43  62  77  ...  64  40  51   

x                                                
y                         3                      
direction            WB  EB  NB  NE  SB  SW  WB  
time                                             
1991-04-01 00:00:00  51  39  64  30  70  29  26  
1991-04-01 00:20:00  58  33  67  30  80  29  57  
1991-04-01 00:40:00  63  33  64  30  79  29  48  
1991-04-01 01:00:00  52  33  33  30  77  29  44  
1991-04-01 01:20:00  50  33  50  30  79  29  44  
...                  ..  ..  ..  ..  ..  ..  ..  
1991-09-30 10:20:00  70  90  53  29  68  22  49  
1991-09-30 10:40:00  68  45  52  25  70  20  47  
1991-09-30 11:00:00  68  44  58  29  69  19  37  
1991-09-30 11:20:00  67  57  59  22  69  22  53  
1991-09-30 11:40:00  71  48  54  28  68  17  24  

[13140 rows x 65 columns]

In [ ]:
train_test_split = datetime.strptime('1991-09-23 12:00:00', '%Y-%m-%d %H:%M:%S')
df_train = imputed_train.loc[imputed_train.index < train_test_split]
df_val = imputed_train.loc[imputed_train.index >= train_test_split]

In [ ]:
sc = MinMaxScaler()
train_sc = sc.fit_transform(df_train)
val_sc = sc.transform(df_val)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,


## LSTM

Long Short Term Memory networks – usually just called “LSTMs” – are a special kind of RNN, capable of learning long-term dependencies.

LSTMs are explicitly designed to avoid the long-term dependency problem. Remembering information for long periods of time is practically their default behavior, not something they struggle to learn! [source: [link](https://colah.github.io/posts/2015-08-Understanding-LSTMs/)]

In [ ]:
# reference: https://towardsdatascience.com/cnn-lstm-based-models-for-multiple-parallel-input-and-multi-step-forecast-6fe2172f7668


def split_sequence(sequence, look_back, forecast_horizon):
    
    X, y = list(), list()
    for i in range(len(sequence)): 
        lag_end = i + look_back
        forecast_end = lag_end + forecast_horizon
        if forecast_end > len(sequence):
            break
        seq_x, seq_y = sequence[i:lag_end], sequence[lag_end:forecast_end]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [ ]:
def inverse_transform(y_test, yhat):
    y_test_reshaped = y_test.reshape(-1, y_test.shape[-1])
    yhat_reshaped = yhat.reshape(-1, yhat.shape[-1])
    yhat_inverse = sc.inverse_transform(yhat_reshaped)
    y_test_inverse = sc.inverse_transform(y_test_reshaped)
    return yhat_inverse, y_test_inverse

In [ ]:
features = len(df_train.columns)
forecast = 36
period = 108

In [ ]:
X_train, y_train = split_sequence(train_sc, look_back=period, forecast_horizon=forecast)
X_val, y_val = split_sequence(val_sc, look_back=period, forecast_horizon=forecast)
rlrop = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, mode='min', patience=3, min_lr=0.001)

In [ ]:
epochs = 15
batch_size = 72
validation = 0.1

In [ ]:
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=9, activation='relu', input_shape=(period, features)))
model.add(Conv1D(filters=64, kernel_size=9, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
# model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
# model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(RepeatVector(forecast))
model.add(LSTM(200, activation='relu', return_sequences=True))
model.add(LSTM(100, activation='relu', return_sequences=True))
# model.add(LSTM(units = 100, return_sequences=True))
model.add(TimeDistributed(Dense(100, activation='relu')))
# model.add(TimeDistributed(Dense(512, kernel_initializer='normal',activation='relu')))
# model.add(TimeDistributed(Dense(100, kernel_initializer='normal',activation='relu')))
model.add(TimeDistributed(Dense(features, kernel_initializer='normal')))
model.compile(loss='mse', optimizer='adam', metrics=['mean_absolute_error'])

In [ ]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_19 (Conv1D)           (None, 100, 64)           37504     
_________________________________________________________________
conv1d_20 (Conv1D)           (None, 92, 64)            36928     
_________________________________________________________________
max_pooling1d_10 (MaxPooling (None, 46, 64)            0         
_________________________________________________________________
flatten_9 (Flatten)          (None, 2944)              0         
_________________________________________________________________
repeat_vector_9 (RepeatVecto (None, 36, 2944)          0         
_________________________________________________________________
lstm_18 (LSTM)               (None, 36, 200)           2516000   
_________________________________________________________________
lstm_19 (LSTM)               (None, 36, 100)          

In [ ]:
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation,callbacks=[rlrop])

Epoch 1/15
157/157 [==============================] - 25s 145ms/step - loss: 1.7963 - mean_absolute_error: 0.1918 - val_loss: 0.0130 - val_mean_absolute_error: 0.0840
Epoch 2/15
157/157 [==============================] - 23s 149ms/step - loss: 0.0114 - mean_absolute_error: 0.0783 - val_loss: 0.0113 - val_mean_absolute_error: 0.0762
Epoch 3/15
157/157 [==============================] - 22s 143ms/step - loss: 0.0106 - mean_absolute_error: 0.0751 - val_loss: 0.0111 - val_mean_absolute_error: 0.0769
Epoch 4/15
157/157 [==============================] - 23s 144ms/step - loss: 0.0103 - mean_absolute_error: 0.0740 - val_loss: 0.0112 - val_mean_absolute_error: 0.0777
Epoch 5/15
157/157 [==============================] - 22s 140ms/step - loss: 0.0101 - mean_absolute_error: 0.0732 - val_loss: 0.0111 - val_mean_absolute_error: 0.0772
Epoch 6/15
157/157 [==============================] - 23s 146ms/step - loss: 0.0100 - mean_absolute_error: 0.0726 - val_loss: 0.0104 - val_mean_absolute_error: 0.073

In [ ]:
mond = model.predict(df_train.iloc[-period:,:].values.reshape(1,period,features)/100, verbose=0)
mae_ = tf.keras.losses.MeanAbsoluteError()
mae = mae_(df_val.iloc[:forecast,:].values,np.round(mond*100).reshape(forecast,features))

mae: tf.Tensor(5.789316, shape=(), dtype=float32)
mse: tf.Tensor(60.013252, shape=(), dtype=float32)


In [ ]:
X_train_f, y_train_f = split_sequence(sc.fit_transform(imputed_train), look_back=period, forecast_horizon=forecast)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,


In [ ]:
history_f = model.fit(X_train_f, y_train_f, epochs=epochs, batch_size=batch_size, validation_split=validation,callbacks=[rlrop])

Epoch 1/15
163/163 [==============================] - 26s 149ms/step - loss: 0.1235 - mean_absolute_error: 0.1541 - val_loss: 0.0123 - val_mean_absolute_error: 0.0809
Epoch 2/15
163/163 [==============================] - 24s 144ms/step - loss: 0.0109 - mean_absolute_error: 0.0761 - val_loss: 0.0106 - val_mean_absolute_error: 0.0744
Epoch 3/15
163/163 [==============================] - 23s 141ms/step - loss: 0.0103 - mean_absolute_error: 0.0736 - val_loss: 0.0103 - val_mean_absolute_error: 0.0730
Epoch 4/15
163/163 [==============================] - 24s 146ms/step - loss: 0.0100 - mean_absolute_error: 0.0724 - val_loss: 0.0102 - val_mean_absolute_error: 0.0723
Epoch 5/15
163/163 [==============================] - 23s 144ms/step - loss: 0.0098 - mean_absolute_error: 0.0715 - val_loss: 0.0103 - val_mean_absolute_error: 0.0723
Epoch 6/15
163/163 [==============================] - 24s 145ms/step - loss: 0.0096 - mean_absolute_error: 0.0707 - val_loss: 0.0103 - val_mean_absolute_error: 0.072

In [ ]:
pred = model.predict(imputed_train.iloc[-period:,:].values.reshape(1,period,features)/100, verbose=0)


In [ ]:
submission = pd.read_csv('../input/tabular-playground-series-mar-2022/sample_submission.csv')
submission['congestion'] = np.round(pred.reshape(36*65,)*100).astype(int)
submission.to_csv('./submission.csv', index=False)

In [ ]:
submission

,row_id,congestion
0,848835,46
1,848836,37
2,848837,49
3,848838,25
4,848839,67
...,...,...
2335,851170,61
2336,851171,33
2337,851172,73
2338,851173,19
